## Criando diretório Bronze

### Criei um volume delta já que no free tier o acesso a mnt/ é negado

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS workspace.resource;

In [0]:
%py
# Criando diretório Bronze
# Definir caminhos base
# BRONZE_PATH = "/mnt/delta/bronze"
BRONZE_PATH = "/Volumes/workspace/default/delta/bronze"
SOURCE_PATH = "dbfs:/databricks-datasets/"

# Criar diretórios se não existirem
dbutils.fs.mkdirs(BRONZE_PATH)

## Verificando os dados de teste disponíveis

In [0]:
display(dbutils.fs.ls(SOURCE_PATH))


### Dataset de vôos

In [0]:
%fs
ls /databricks-datasets/flights

### Consultando os arquivos e salvando em delta parquet

In [0]:
# lendo o readme
readme_path = 'dbfs:/databricks-datasets/flights/README.md'
readme = spark.read.text(readme_path)
display(readme)

In [0]:
# Caminho do arquivo
file_path = 'dbfs:/databricks-datasets/flights/departuredelays.csv'

# Leitura do arquivo CSV
df_fly = spark.read.format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .option('delimiter', ',') \
    .load(file_path)

# Exibir o DataFrame
display(df_fly)

In [0]:
#     .partitionBy("Registro_voos") \
# .saveAsTable("registro_voos") registra a tabela no catalogo
# mas não pe uma boa ideia na camada bronze

# salvando a tabela na camada bronze
# cada tabela deve ter seu proprio diretorio
df_fly \
    .write.format("delta") \
    .option("mergeSchema", "true") \
    .mode("overwrite") \
    .save(f"{BRONZE_PATH}/flight/registro_voos")


In [0]:

# leitura do arquivo txt
file_path_fly_code = f"{SOURCE_PATH}/flights/airport-codes-na.txt"

df_fly_code = spark.read \
    .option("header", "true") \
    .option("sep", "\t") \
    .option("inferSchema", "true") \
    .option("ignoreLeadingWhiteSpace", "true") \
    .option("ignoreTrailingWhiteSpace", "true") \
    .csv(file_path_fly_code)
# o metodo .csv funciona para qualquer arquivo com um delimitador
# Exibir o DataFrame
display(df_fly_code)

In [0]:
# salvando a tabela na camada bronze
df_fly_code \
    .write.format("delta") \
    .option("mergeSchema", "true") \
    .mode("overwrite") \
    .save(f"{BRONZE_PATH}/flight/codigo_voos")
